# 📘 Використання Azure OpenAI Function Calling

## Вступ

Цей ноутбук демонструє, як використовувати режим **Function Calling** у Microsoft Azure OpenAI з Python SDK `azure.ai.inference`.

Ми розглянемо:
- Що таке Function Calling та його варіанти використання
- Як описати функцію у форматі OpenAI Function Schema
- Як передати її як інструмент (tool) до моделі
- Як обробити відповідь, якщо модель хоче викликати функцію
- Як відповісти від імені функції, щоб модель могла продовжити відповідь
- Як інтегрувати Function Calling в реальний застосунок

> ⚠️ Перед запуском переконайтесь, що у вас є:
> - 🔑 API ключ Azure OpenAI
> - 🌐 Endpoint
> - 📦 Деплоймент моделі GPT (наприклад, `gpt-4`, `gpt-35-turbo`)

## Навчальні цілі

Після завершення цього уроку ви:
- Зрозумієте призначення та переваги Function Calling
- Навчитеся налаштовувати Function Calling у сервісі Azure OpenAI
- Зможете розробляти ефективні функціональні виклики для своїх застосунків
- Зрозумієте повний цикл обробки функціональних викликів

## Розуміння Function Calling

У цьому уроці ми створимо функціонал для освітнього стартапу, який дозволяє користувачам використовувати чат-бота для пошуку технічних курсів. Ми будемо рекомендувати курси, що відповідають їхньому рівню навичок, поточній ролі та технології, яка їх цікавить.

Для цього ми використовуємо комбінацію:
- **Azure OpenAI** для створення інтерактивного чат-досвіду
- **Microsoft Learn Catalog API** для пошуку курсів на основі запиту користувача
- **Function Calling** для обробки запиту користувача та передачі до функції, яка зробить API-запит

### Навіщо використовувати Function Calling?

Якщо ви вже працювали з великими мовними моделями (LLM), ви, напевно, розумієте їх потужність, а також їх обмеження.

Function Calling - це функціональність сервісу Azure OpenAI, яка допомагає подолати такі обмеження:
1. **Послідовний формат відповіді** - отримання структурованих даних
2. **Можливість використовувати дані з інших джерел** у контексті чату

До появи Function Calling відповіді від LLM були неструктурованими та непослідовними. Розробникам доводилося писати складний код валідації, щоб обробляти різні варіанти відповідей.

Користувачі не могли отримати відповіді на запитання типу "Яка зараз погода у Києві?", оскільки моделі обмежені даними, на яких вони навчались.

### Варіанти використання Function Calling

**1. Виклик зовнішніх інструментів**
Чат-боти чудово відповідають на запитання користувачів. Використовуючи Function Calling, чат-боти можуть використовувати повідомлення від користувачів для виконання певних завдань. Наприклад, студент може попросити чат-бота: "Надішли імейл моєму викладачу, що мені потрібна додаткова допомога з цього предмета". Це може викликати функцію `send_email(to: string, body: string)`.

**2. Створення запитів до API або бази даних**
Користувачі можуть знаходити інформацію, використовуючи природну мову, яка перетворюється на форматований запит. Приклад: викладач запитує "Хто з студентів виконав останнє завдання?", що може викликати функцію `get_completed(student_name: string, assignment: int, current_status: string)`.

**3. Створення структурованих даних**
Користувачі можуть взяти блок тексту або CSV і використовувати LLM для вилучення важливої інформації. Наприклад, студент може перетворити статтю з Вікіпедії про мирні угоди для створення AI-карток. Це можна зробити за допомогою функції `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`.

## 🔐 Крок 1: Підключення до Azure OpenAI

Перший крок – ініціалізувати клієнт `ChatCompletionsClient`, вказавши endpoint та ключ доступу.

In [18]:
import os
import json
import requests
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import ChatCompletionsToolDefinition
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

# Виберіть модель загального призначення для тексту
deployment = "gpt-4o"


## 🔧 Крок 2: Визначення функції

Функцію описуємо у форматі JSON Schema (OpenAI Function Calling format) і передаємо як `ChatCompletionsToolDefinition`.

> У нашому випадку, функція `search_courses` — імітує пошук курсів з документації Microsoft Learn за заданими критеріями.

### Елементи функціонального виклику

Ось пояснення ключових частин функціонального виклику:

- **name** - Ім'я функції, яку ми хочемо викликати
- **description** - Опис того, як працює функція. Важливо бути конкретним і чітким
- **parameters** - Список значень та форматів, які модель повинна використовувати у відповіді
- **type** - Тип даних властивостей
- **properties** - Список конкретних значень, які модель буде використовувати для відповіді
- **required** - Обов'язкові властивості для виконання функціонального виклику

In [20]:
# 🔧 Визначення функції, яку модель може викликати
functions = [
    ChatCompletionsToolDefinition(
        type="function",
        function={
            "name": "search_courses",
            "description": "Returns a list of training courses from the Microsoft catalog",
            "parameters": {
                "type": "object",
                "properties": {
                    "role": {
                        "type": "string",
                        "description": """User role (for example: developer, student)"""
                    },
                    "product": {
                        "type": "string",
                        "description": "Covered product (Azure, Power BI, etc.)"
                    },
                    "level": {
                        "type": "string",
                        "description": "User experience level"
                    }
                },
                "required": ["role"]
            }
        }
    )
]


## 📤 Крок 3: Надсилання повідомлення користувача

Ми надсилаємо запит від імені користувача (наприклад, "знайди курси для початківця-розробника по Azure"), і модель вирішує, чи слід викликати функцію.

Щоб інтегрувати функцію у виклик Chat Completion API, ми додаємо `tools=functions` до запиту. Встановлення `tool_choice="auto"` дозволяє LLM самостійно вирішити, яку функцію викликати, виходячи з повідомлення користувача.

In [21]:
# 🧠 Створення повідомлення та виклик моделі з інструментами
messages = [
    {
        "role": "user",
        "content": "Find a course for a beginner developer on Azure"
    }
]

response = client.complete(
    model=deployment,
    messages=messages,
    tools=functions,
    tool_choice="auto"
)

response_message = response.choices[0].message
print("📥 Відповідь моделі:")
print(response_message)


📥 Відповідь моделі:
{'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'function': {'arguments': '{"role":"developer","product":"Azure","level":"beginner"}', 'name': 'search_courses'}, 'id': 'call_IS7W2kYaQBMWyCDtKmfJtWMF', 'type': 'function'}]}


## ⚙️ Крок 4: Обробка виклику функції (tool_call)

Якщо модель вирішила викликати функцію, ми зчитуємо її аргументи, викликаємо локальну функцію `search_courses`, і передаємо результат назад в модель як `"role": "tool"`.

Після цього модель формує підсумкову відповідь з урахуванням виклику.

### Інтеграція у застосунок

Для інтеграції у реальний застосунок необхідно:
1. Перевірити, чи модель хоче викликати функцію
2. Отримати аргументи з відповіді моделі
3. Викликати відповідну функцію з отриманими аргументами
4. Додати відповідь від функції до історії повідомлень
5. Зробити новий запит до моделі для отримання підсумкової відповіді користувачу

In [24]:
# ⚙️ Обробка function_call (tool_call)
tool_calls = response_message.tool_calls

if tool_calls and len(tool_calls) > 0:
    first_tool_call = tool_calls[0]
    function_name = first_tool_call.function.name
    function_args = json.loads(first_tool_call.function.arguments)

    def search_courses(role, product, level):
        url = "https://learn.microsoft.com/api/catalog/"
        params = {
            "role": role,
            "product": product,
            "level": level
        }
        response = requests.get(url, params=params)
        modules = response.json().get("modules", [])
        results = []
        for module in modules[:5]:
            title = module.get("title")
            url = module.get("url")
            results.append({"title": title, "url": url})
        return json.dumps(results, ensure_ascii=False)
        

    available_functions = {
        "search_courses": search_courses,
    }

    function_to_call = available_functions[function_name]
    function_response = function_to_call(**function_args)

    print("✅ Результат виклику функції:")
    print(function_response)

    # Додаємо відповідь моделі з tool_calls до історії повідомлень
    messages.append({
        "role": "assistant",
        "content": "Відповідай українською",
        "tool_calls": [
            {
                "id": first_tool_call.id,
                "type": "function",
                "function": {
                    "name": first_tool_call.function.name,
                    "arguments": first_tool_call.function.arguments
                }
            }
        ]
    })

    # Додаємо відповідь функції
    messages.append({
        "role": "tool",  # У найновіших версіях API використовується "tool" замість "function"
        "tool_call_id": first_tool_call.id,  # Це критично!
        "name": function_name,
        "content": function_response
    })

    # Отримуємо фінальну відповідь від моделі
    next_response = client.complete(
        model=deployment,
        messages=messages
    )

    print("📤 Остаточна відповідь моделі:")
    print(next_response.choices[0].message.content)
else:
    print("⚠️ Модель не захотіла викликати функцію.")


✅ Результат виклику функції:
[{"title": "Introduction to audio classification with TensorFlow", "url": "https://learn.microsoft.com/en-us/training/modules/intro-audio-classification-tensorflow/?WT.mc_id=api_CatalogApi"}, {"title": "Design a Performant Data Model in Azure SQL Database with Azure Data Studio", "url": "https://learn.microsoft.com/en-us/training/modules/design-a-data-model-with-ads/?WT.mc_id=api_CatalogApi"}, {"title": "Develop products with screen reader support", "url": "https://learn.microsoft.com/en-us/training/modules/develop-products-with-screen-reader-support/?WT.mc_id=api_CatalogApi"}, {"title": "Application scalability on AKS with HorizontalPodAutoscalers", "url": "https://learn.microsoft.com/en-us/training/modules/aks-application-autoscaling-native/?WT.mc_id=api_CatalogApi"}, {"title": "Host a web application with Azure App Service", "url": "https://learn.microsoft.com/en-us/training/modules/host-a-web-app-with-azure-app-service/?WT.mc_id=api_CatalogApi"}]
📤 Оста

## Висновки та практичні завдання

Тепер ви знаєте, як інтегрувати Function Calling у ваші застосунки з Azure OpenAI. Ця потужна функціональність дозволяє створювати більш інтелектуальні та корисні взаємодії з користувачами.

### Завдання для подальшого вивчення:

1. Додайте більше параметрів до функції, які можуть допомогти учням знаходити більше курсів. Перегляньте доступні API параметри [тут](https://learn.microsoft.com/training/support/catalog-api-developer-reference).
2. Створіть інший функціональний виклик, який отримує додаткову інформацію від користувача, наприклад, його рідну мову.
3. Додайте обробку помилок для випадків, коли функціональний виклик та/або API-виклик не повертає відповідних курсів.
4. Розширте функціонал для рекомендації не лише курсів, але й навчальних шляхів (learning paths).